<center><img alt="title" src="./assets/TFT.jpg"/></center>

# <center>Predicting the Best Strategy to Win in Teamfight Tactics</center>

**<center>Jason Dang, Nhat-Nguyen Vo, Wisely Kong</center>**

## <center>Introduction</center>

Teamfight Tactics (TFT) is an auto battler game developed by Riot Games. Auto battlers are strategy based games in which the player is given preparation time in order to assemble a team on a chess-like board (we'll just call these teams boards) with various units in order to fight an opposing player's board. These fights between boards do not receive any additional input from its players, as the units automatically move and attack until the enemy's board has no units.

Since TFT is a rather complex strategy game, there a variety of concepts to understand about the game.

In the game of TFT, there are 8 players per game. The main objective in each game is to be among the last 4 players left alive, as the game considers "Top 4" to be a moral victory. But in order to truly win a game of TFT, a player must obviously be the last player left alive. The main factors that go into a player's game are: their Health, Gold, Level, Units, Augments, and Components/Items.

- Health: Each player starts off with 100 Health. When a player loses a battle, their health is reduced based on how long the game has lasted and how many units are remaining on the board. At 0 Health, the player is defeated.
- Gold: During each preparation phase, the player is given Gold to utilize in order to make their board as strong as possible. Players can spend their Gold on either Levels or Units in order to do so.
- Level: The amount of units a player can place on their board is dependent on their current level. A player at level 3 can only place 3 units, level 4 can only place 4 units, and so on. The player naturally levels up as the game progresses but can also utilize their Gold during the preparation phase to level up faster.
- Units: During the preparation phase before facing against an opposing player's board, the player is presented a "Shop" with 5 Units in which they can utilize their Gold in order to purchase and place on their board for the next fight. The player can choose to use their Gold instead to re-roll the Shop to choose from 5 different choices. The Units in the Shop have three important aspects: Traits, Cost, and Rarity
    - Traits: Units have "Traits" that grant special effects for Unit with the Trait, but these effects can only be activated when a certain number of unique Units (with the Trait) are placed on the board at the same time. The more Units of the same trait, the stronger the effect.
    - Cost: When Units appear in the Shop, they cost a certain amount of Gold to purchase. The amount a Unit costs relates to how powerful the Unit is, where a Unit costing 1 Gold (the lowest amount) is comparatively weaker to a Unit costing 5 Gold (the highest amount). In addition, the Cost of a Unit relates to how often they would appear in a Shop, where a 1 Gold Unit will appear more often than a 5 Gold Unit.
    - Rarity: When first purchased from the Shop, Units starts at "1-Star". When 3 of the same Unit is purchased the Unit upgrades into "2-Star", in which the Unit's strength is increased. When 3 "2-Star" Units (of the same type) are made, the Unit becomes a "3-Star". The likelihood of reaching "3-Star" is more likely for lower Cost Units (1, 2, 3 Cost Units) compared to higher Cost Units (4, 5 Cost Units) due to how often lower Cost Units appear in the Shop.
- Augments: In certain intervals of the game players are presented with 3 options that provides a variety of benefits to the player, by either making certain Traits stronger or being granted multiple 5 Cost Units for free. There are 3 tiers of Augments, rising from lower strength to higher strength: Silver, Gold, Prismatic. Players can only have 3 Augments per game.
- Components/Items: In certain intervals of the game, players are granted materials, also known as Components, in which they can place on their Units to make them stronger. When two Components are combined (by placing both onto a unit), an Item is created based on the combined components that grants special effects for that specific Unit. A Unit can only have a maximum combination of 3 Items/Components and cannot be interchanged once placed. Ideally, the most important Units on a player's board will always receive 3 Items.

More information on the game can be found [here](https://tftactics.gg/).

<center><img alt="ExampleTFT" src="./assets/TFT_Example.png" title="TFT Concepts in Game"/></center>

<center><img alt="ExampleAugments"  src="./assets/Augments.png" title="Augment Example"/></center>

In this tutorial...

## <center>Data Collection</center>

In order to predict the best strategy in order to win in TFT, we must first decide where we will be retrieving our data. We decided the best source of data would come from the best players in the game, as they are considered the best for a reason: they know how to win. In TFT, the top players at a given time can be located through [this link, showing the leaderboard for the top players in North America](https://lolchess.gg/leaderboards?mode=ranked&region=na). As for the contents of the data, we arbitrarily chose to select the top 100 players and their last 75 games played. Since the leaderboard is constantly changing every few minutes, as players move up and down the leaderboard, we uploaded a compiled dataset to [Kaggle](https://www.kaggle.com/datasets/cmsc320tftproject/cmsc320-tft-challenger-dataset-set-65) to ease the replication process of this tutorial and have to have the data be consistent with what we use. But for clarity sakes, we will showcase the data scraping process.

The first step is gathering the names of those within the Top 100 rankings, based on the leaderboard website. The next step is to utilize the Riot Games API in order to extract the necessary data, that being the most recent 75 games those 100 players have played. In order to access the Riot Games API, you must [create / login](https://developer.riotgames.com/) to a Riot Games account to generate a development API key. Again, downloading the dataset from Kaggle will allow you to avoid having to deal with creating a development API key. Once the names are gathered and an API Key is obtained, the final step would be to compile and tidy the relevant data into a table.

In [32]:
# Import the necessary libraries that will be used for the tutorial
import requests
import csv
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ast import literal_eval

In [ ]:
# Obtain the HTML for the leaderboard website in order to selectively obtain the names of those in the Top 100.
leaderboard = requests.get("https://lolchess.gg/leaderboards?mode=ranked&region=na")
# Using the library BeautifulSoup, parse the HTML to obtain the required information easier (the names of the Top 100)
soup = BeautifulSoup(leaderboard.content, 'html.parser')

In [ ]:
# Create an array in order to hold the names
playerList = []

# Limit the HTML to show just the relevant sections (the names)
table_body=soup.find('tbody')
rows = table_body.find_all('tr')

# Iterate through the HTML and store specifically the names of the player into the array.
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    # Separate irrelevant text from the player's name with .partition()
    playerList.append(cols[1].partition("\n\n"))

# Need to remove the irrelevant text that resulted from the previous .partition() call
cleanList = []
for currentPlayer in playerList:
    player = (currentPlayer[2].lstrip())
    cleanList.append(player);

In [ ]:
# This is the API Key required to utilize Riot Games API. These expire 24 hours after being generated, so this will need to be updated if you want to progress with the code. 
# Again, the compiled dataset is in the Kaggle link if you want to avoid creating a Riot Games account. 
key = "RGAPI-792fade4-60a0-4bc5-968d-2cacf97fe150"
# Each player has a unique ID that is recognized by the Riot Games API. With this ID, we can now utilize the API to obtain the last 75 matches of that particular ID.
puuidList = []

# Iterate through the player's name in order to obtain their PUUID.
for player in cleanList:
    url = "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/" + player + "?api_key=" + key
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    jsonConvert = json.loads(str(soup))
    
    try:
        puuid = jsonConvert['puuid']
        puuidList.append(puuid)
    except KeyError:
        # Calling the API with too many commands could result in a Rate Limit Exceeded Error, which might result in losing data for that specific player.
        # If experiencing the Rate Limit Exceeded Error, wait to run the code again. 
        print("Unable to get " + player + " data")

In [ ]:
# Use a dictionary to store the last 75 matches per player.
matches = {}

for current in puuidList:
    url = "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + current + "/ids?count=75&api_key=" + key
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    jsonConvert = json.loads(str(soup))
    matches[current] = jsonConvert

listToSet = []

#Take the dictionary and convert to a list. Then convert that to a set to create a unique list of matches for the dataset, to remove duplicate matches
for puuid in matches:
    currentMatchList = matches[puuid]
    for currentMatch in currentMatchList:
        listToSet.append(currentMatch)

listToSet = list(set(listToSet))

In [ ]:
# Create the dataframe for the data, that will display the relevant information from a match
df = pd.DataFrame();

# Since the Riot API limits the amount of requests you can send, you'd have to change the range of the for loop in order to extract all of the matches (i.e. change the [0:10] to a different range of values).
# This process does take a bit of time, mostly due to waiting for the ability to send more requests
# The rest of the tutorial will use the aforementioned CSV file from the Kaggle link in order to have consistent data.
for match in listToSet[0:10]:
    url = "https://americas.api.riotgames.com/tft/match/v1/matches/" + match + "?api_key=" + key
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    soup = str(soup)
    soup = soup.replace("\">\"", "")
    soup = soup.replace("</releases>", "")
    jsonConvert = json.loads(str(soup))
    data = pd.DataFrame.from_dict(jsonConvert['info']['participants'])
    # These dropped columns are irrelevant to the tutorial
    data = data.drop('companion', axis = 1)
    data = data.drop('gold_left', axis = 1)
    data = data.drop('last_round', axis = 1)
    data = data.drop('players_eliminated', axis = 1)
    data = data.drop('time_eliminated', axis = 1)
    data = data.drop('puuid', axis = 1)
    data = data.drop('total_damage_to_players', axis = 1)
    data['Match ID'] = jsonConvert['metadata']['match_id']
    df = pd.concat([df, data], axis = 0)

In [ ]:
# The dataset from the CSV, that shows the results and relevant data of all of the matches 
tutorialDataset = pd.read_csv('TFTdataset.csv')
tutorialDataset

,augments,level,placement,traits,units,Match ID
0,"['DebonairTrait', 'Distancing2', 'DebonairEmbl...",7,8,"[{'name': 'Assassin', 'num_units': 2, 'style':...","[{'character_id': 'Talon', 'items': [69, 19, 2...",NA1_4247681350
1,"['AssassinCutthroat', 'RichGetRicher', 'Divers...",9,3,"[{'name': 'Arcanist', 'num_units': 2, 'style':...","[{'character_id': 'Kassadin', 'items': [], 'na...",NA1_4247681350
2,"['EnchanterTrait', 'SunfireBoard', 'TwinshotTr...",8,2,"[{'name': 'Bruiser', 'num_units': 2, 'style': ...","[{'character_id': 'Sejuani', 'items': [], 'nam...",NA1_4247681350
3,"['CalculatedLoss', 'ExperimentalEmblem', 'Arca...",9,1,"[{'name': 'Arcanist', 'num_units': 4, 'style':...","[{'character_id': 'Kassadin', 'items': [], 'na...",NA1_4247681350
4,"['Phalanx1', 'ArcanistSpellBlade', 'Experiment...",9,4,"[{'name': 'Arcanist', 'num_units': 2, 'style':...","[{'character_id': 'RekSai', 'items': [], 'name...",NA1_4247681350
...,...,...,...,...,...,...
39995,"['Disintegrator1', 'TomeOfTraits1', 'AssassinE...",9,1,"[{'name': 'Assassin', 'num_units': 4, 'style':...","[{'character_id': 'Zilean', 'items': [], 'name...",NA1_4255593521
39996,"['SocialiteDuet', 'Disintegrator1', 'Backfoot2']",8,3,"[{'name': 'Bodyguard', 'num_units': 1, 'style'...","[{'character_id': 'Corki', 'items': [], 'name'...",NA1_4255593521
39997,"['ItemGrabBag1', 'ThrillOfTheHunt1', 'JeweledL...",8,4,"[{'name': 'Assassin', 'num_units': 4, 'style':...","[{'character_id': 'Nocturne', 'items': [], 'na...",NA1_4255593521
39998,"['Featherweights1', 'Weakspot', 'Archangel2']",8,7,"[{'name': 'Arcanist', 'num_units': 2, 'style':...","[{'character_id': 'Poppy', 'items': [], 'name'...",NA1_4255593521


Within the compiled CSV/dataset, each row represents a player. The columns are:  
- Augments: The three augments the player ended a game with
- Level: The level the player ended a game with
- Placement: The player's overall placement at the end of the game
- Traits: The active Traits the player had by the end of the game 
- Units: The Units the player had by the end of the game (with their rarities and items)   
- Match ID: The ID of the game. 

# <center>Data Management</center>

With the csv file created we can now clean it to make the dataset more readable.

As of now each row of the csv will look something like this:

<center>"augments": "['TFT6_Augment_DebonairTrait', ...]",

"level": "8",

"placement": "8",

"traits": "[{'name': 'Set6_traitname', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, ...]",

"units": "[{'character_id': 'TFT6_(CharacterName))', 'itemNames': ['TFT_Item_(ItemName)', ...], 'items': [(ItemID), ...], 'name': '(CharacterName)', 'rarity': (Rarity number), 'tier': (Unit Cost))}, ...]",

"Match ID": "Match ID" </center>


Of the information available there's alot of redundant and useless headers such as "TFT6\_" and "Set6\_" or "TFT\_Item\_". For the sake of readability we shall remove them from the csv.

To do so we can create a function that finds these exact phrases in the csv and simply erases them.

In [ ]:
# var filePath: str, contains the filepath that leads to the csv file
def cleanCSV(filePath):
    # open the csv file
    f = open(filePath)
    # save text in the file to a string to edit
    text = f.read()
    #  close the csv file
    f.close()

    # the next 6 lines of code deletes the specific unnecesary phrases
    # as well as an error item id that sometimes appears in collected data from our csv
    text = text.replace('TFT6_', '')
    text = text.replace('Set6_', '')
    text = text.replace('TFT_Item_', '')
    text = text.replace('Augment_', '')
    text = text.replace(', 10006', '')

    # open the csv file again
    f = open(filePath,'w')

    # save new clean text to the csv file
    f.write(text)

    # close the csv file
    f.close()

# run the function which will overwrite the old dataset with the cleaned dataset
cleanCSV("TFTdataset.csv")


The CSV is now cleaned of useless headers. We will now convert the csv to a JSON to make the data set easier to work with as well as give each row a unique ID.

Once again we'll make a function for this as well

In [6]:
# var csvFilePath: str, contains the filepath that leads to the csv file
# var jsonFilePath: str, place to save the JSON file
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary called data
    data = {}
    # create a counter to number each row of the csv 
    counter = 0
    # Open the csv with a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary and add it to data
        for rows in csvReader:
             
            # save the row to the key represented by the counter
            data[counter] = rows
            # increment the counter
            counter += 1
 
    # Open a json writer, and use the json.dumps() function to convert the dictionary into a Json object
    # save the JSON object to a file using write()
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

# run the function which will create a json file version of the csv file
make_json("TFTdataset.csv", 'TFTdataset.json')

#  open the json file and save it to a variable for use later using json.load()
with open('TFTdataset.json') as json_file:
    data = json.load(json_file)

We we now convert all the item IDs into the English names of the Items to make the data easier to read, to do so
We will be needing one more thing and thats a itemDictionary or a dictionary for converting item IDs to items names.

These can be found by searching through the en_us.json file found at https://raw.communitydragon.org/latest/cdragon/tft/

You can either manually skim through the file for item codes or create a code to clean this, however for simplicity's sake the finished item dictionary is already saved to a json file and loaded into the code below.

In [45]:
with open('TFTitemDictionary.json') as json_file:
    itemDictionary = json.load(json_file)

With a JSON file and an item dictionary, we can now replace all item ID numbers with Item Names as well as remove redundant sections such as "itemName" and "name".

In [8]:

# create a list containing the all row numbers (keys) of the dataset
keys = list(data.keys())

# iterate through the list of keys
for key in keys:
    # save the units section of the row specified by key
    units = data[key]['units']

    # Units will be in the form of a list of dictionaries representing each unit in the team within a string
    # We want string to be just a list
    # To do so we will first make sure that units is a string
    
    if(units.__class__.__name__ == 'str'):
        if len(units) < 2:
            print("error line: ", key)
        # convert the string into a list of dictionaries using literal_eval()
        units = literal_eval(units)

    # For every item in every unit within the row we replace each ID with its respective name within the itemDictionary
    for char in range(len(units)):
        info = list(units[char].keys())
        # if this unit has a "itemNames" section remove it
        if 'itemNames' in info:
            units[char].pop('itemNames')

        # if this unit has a "names" section remove it
        if 'names' in info:
            units[char].pop('name')
        for item in range(len(units[char]['items'])):
            
            # Confirm that the ID is an actual Item ID by making sure its length is less or equal to 4
            # This is for the case that the item name is already there which could be the case
            if(len(str(units[char]['items'][item])) <= 4):
                units[char]['items'][item] = itemDictionary[str(units[char]['items'][item])]
    
    # update the row with the new information
    data[key]['units'] = units

# Save the dictionary to the json file
with open('TFTdataset.json', 'w') as json_file:
    json.dump(data, json_file)
    

We can now convert the json directly into a pandas dataframe for analysis.


In [9]:
# if the JSON isn't already saved to a variable in the form of a dictionary
# you will first need to open the json file which can be done with the following 2 lines of commented code 

# with open('TFTdataset.json') as json_file:
#     data = json.load(json_file)

df = pd.DataFrame.from_dict(data, orient = 'index')

# Some columns are currently occupied by arrays in the form of strings instead of just arrays
# To fix this you can use the following code that simply runs eval() on each array string and turns them into regular arrays
df["augments"] = df["augments"].apply(eval)
df["traits"] = df["traits"].apply(eval)
df["placement"] = pd.to_numeric(df["placement"])

df

,augments,level,placement,traits,units,Match ID
0,"[DebonairTrait, Distancing2, DebonairEmblem2]",7,8,"[{'name': 'Assassin', 'num_units': 2, 'style':...","[{'character_id': 'Talon', 'items': ['Quicksil...",NA1_4247681350
1,"[AssassinCutthroat, RichGetRicher, Diversify3]",9,3,"[{'name': 'Arcanist', 'num_units': 2, 'style':...","[{'character_id': 'Kassadin', 'items': [], 'na...",NA1_4247681350
2,"[EnchanterTrait, SunfireBoard, TwinshotTrait2]",8,2,"[{'name': 'Bruiser', 'num_units': 2, 'style': ...","[{'character_id': 'Sejuani', 'items': [], 'nam...",NA1_4247681350
3,"[CalculatedLoss, ExperimentalEmblem, ArcanistE...",9,1,"[{'name': 'Arcanist', 'num_units': 4, 'style':...","[{'character_id': 'Kassadin', 'items': [], 'na...",NA1_4247681350
4,"[Phalanx1, ArcanistSpellBlade, ExperimentalEmb...",9,4,"[{'name': 'Arcanist', 'num_units': 2, 'style':...","[{'character_id': 'RekSai', 'items': [], 'name...",NA1_4247681350
...,...,...,...,...,...,...
39995,"[Disintegrator1, TomeOfTraits1, AssassinEmblem]",9,1,"[{'name': 'Assassin', 'num_units': 4, 'style':...","[{'character_id': 'Zilean', 'items': [], 'name...",NA1_4255593521
39996,"[SocialiteDuet, Disintegrator1, Backfoot2]",8,3,"[{'name': 'Bodyguard', 'num_units': 1, 'style'...","[{'character_id': 'Corki', 'items': [], 'name'...",NA1_4255593521
39997,"[ItemGrabBag1, ThrillOfTheHunt1, JeweledLotus]",8,4,"[{'name': 'Assassin', 'num_units': 4, 'style':...","[{'character_id': 'Nocturne', 'items': [], 'na...",NA1_4255593521
39998,"[Featherweights1, Weakspot, Archangel2]",8,7,"[{'name': 'Arcanist', 'num_units': 2, 'style':...","[{'character_id': 'Poppy', 'items': [], 'name'...",NA1_4255593521


# <center>Exploratory Data Analysis</center>


Scatterplot: Traits vs winrate

Scatterplot: augments vs winrate

Scatterplot: Items vs winrate

Scatterplot: champs/unit vs winrate

(remove this later)


In [33]:
def data_vs_winrate(data, group, aggregate_funcs):
    data["wins"] = np.where(data["placement"] <= 4, 1, 0)
    data["losses"] = np.where(data["placement"] >= 5, 1, 0)
    data["games"] = 1

    data = data.groupby([group]).aggregate(aggregate_funcs)
    data["winrate"] = data["wins"] / data["games"]

    return data

In [34]:
def get_weighted_winrate(data):
    weighted_constant = 12.5
    numer = data["wins"] + weighted_constant * data['winrate'].mean()
    denom = data['games'] + weighted_constant
    return numer / denom

In [35]:
augment_df = df.explode("augments")

aggregate_funcs = {'wins': 'sum', 'losses': 'sum', 'games': 'count'}
augment_df = data_vs_winrate(augment_df, 'augments', aggregate_funcs)

augment_df['weighted_winrate'] = get_weighted_winrate(augment_df)

augment_df

,wins,losses,games,winrate,weighted_winrate
augments,,,,,
ArcanistEmblem,349,354,703,0.496444,0.496552
ArcanistEmblem2,88,90,178,0.494382,0.494922
ArcanistRunicShield1,177,200,377,0.469496,0.470559
ArcanistRunicShield2,212,257,469,0.452026,0.453339
ArcanistSpellBlade,253,211,464,0.545259,0.544140
...,...,...,...,...,...
Weakspot3,253,193,446,0.567265,0.565502
Windfall,163,246,409,0.398533,0.401619
WoodlandCharm,22,32,54,0.407407,0.425303


In [47]:
for augment in augment_df.index:
    print(f'"{augment}": "Gold",')

"ArcanistEmblem": "Gold",
"ArcanistEmblem2": "Gold",
"ArcanistRunicShield1": "Gold",
"ArcanistRunicShield2": "Gold",
"ArcanistSpellBlade": "Gold",
"ArcanistTrait": "Gold",
"Archangel2": "Gold",
"Ascension": "Gold",
"AssassinCutthroat": "Gold",
"AssassinEmblem": "Gold",
"AssassinEmblem2": "Gold",
"AssassinSmokeBomb": "Gold",
"AssassinTrait": "Gold",
"Backfoot1": "Gold",
"Backfoot2": "Gold",
"Backfoot3": "Gold",
"BandOfThieves1": "Gold",
"Battlemage1": "Gold",
"Battlemage2": "Gold",
"Battlemage3": "Gold",
"BinaryAirdrop": "Gold",
"BlueBattery1": "Gold",
"BlueBattery2": "Gold",
"BrawlerHPRegen1": "Gold",
"BruteEmblem": "Gold",
"BruteEmblem2": "Gold",
"BruteTrait": "Gold",
"CalculatedLoss": "Gold",
"CelestialBlessing1": "Gold",
"CelestialBlessing2": "Gold",
"CelestialBlessing3": "Gold",
"ChallengerEmblem": "Gold",
"ChallengerEmblem2": "Gold",
"ChallengerEnGarde": "Gold",
"ChallengerForAllUnits": "Gold",
"ChallengerTrait": "Gold",
"ChemtechEmblem": "Gold",
"ChemtechEmblem2": "Gold",
"Chemte

In [36]:
trait_df = df.explode("traits")

trait_df["traits"] = trait_df["traits"].apply(lambda x: (x.get('name') + str(x.get('tier_current'))) if type(x) is dict else np.nan)
trait_df = trait_df.dropna(subset=['traits'])

trait_df = data_vs_winrate(trait_df, 'traits', aggregate_funcs)
trait_df['weighted_winrate'] = get_weighted_winrate(trait_df)

trait_df

,wins,losses,games,winrate,weighted_winrate
traits,,,,,
Arcanist0,3354,3396,6750,0.496889,0.496977
Arcanist1,2517,2226,4743,0.530677,0.530713
Arcanist2,774,1053,1827,0.423645,0.424467
Arcanist3,1038,1156,2194,0.473108,0.473513
Arcanist4,527,283,810,0.650617,0.649005
...,...,...,...,...,...
Twinshot4,185,96,281,0.658363,0.653514
Yordle0,6873,6972,13845,0.496425,0.496468
Yordle1,775,1213,1988,0.389839,0.390806


In [37]:
unit_df = df.explode("units")

unit_df["cost"] = unit_df["units"].apply(lambda x: x.get('rarity') + 1 if type(x) is dict else x)
unit_df["units"] = unit_df["units"].apply(lambda x: x.get('character_id') if type(x) is dict else x)

unit_aggregate_functions = {'wins': 'sum', 'losses': 'sum', 'games': 'count', 'cost': 'max'}
unit_df = data_vs_winrate(unit_df, 'units', unit_aggregate_functions)

unit_df['weighted_winrate'] = get_weighted_winrate(unit_df)

unit_df

,wins,losses,games,cost,winrate,weighted_winrate
units,,,,,,
Ahri,3275,3303,6578,4.0,0.497872,0.497897
Alistar,2051,1767,3818,4.0,0.537192,0.537107
Ashe,600,809,1409,2.0,0.425834,0.426583
Blitzcrank,4696,4379,9075,2.0,0.517466,0.517457
Brand,1181,1469,2650,1.0,0.445660,0.445967
Braum,6980,6672,13652,4.0,0.511280,0.511280
Caitlyn,591,670,1261,1.0,0.468676,0.469092
Camille,2640,2662,5302,1.0,0.497925,0.497956
ChoGath,852,954,1806,3.0,0.471761,0.472031


In [38]:
item_df = df.explode("units")

item_df["items"] = item_df["units"].apply(lambda x: x.get('items') if type(x) is dict and len(x.get('items')) == 3 else np.nan)
item_df = item_df.dropna(subset=['items'])
item_df = item_df.explode('items')

item_df = data_vs_winrate(item_df, 'items', aggregate_funcs)
item_df['weighted_winrate'] = get_weighted_winrate(item_df)

item_df

,wins,losses,games,winrate,weighted_winrate
items,,,,,
Anima Visage,143,117,260,0.550000,0.552959
Arcanist Emblem,789,459,1248,0.632212,0.632036
Archangel's Staff,1624,845,2469,0.657756,0.657538
Assassin Emblem,721,406,1127,0.639752,0.639475
B.F. Sword,709,548,1257,0.564041,0.564538
...,...,...,...,...,...
Zenith Edge,14,7,21,0.666667,0.647201
Zephyr,1108,600,1708,0.648712,0.648463
Zhonya's Paradox,86,67,153,0.562092,0.566050
